# **Finding Lane Lines on the Road** 

---

## 1. Steps to determine lane lines

My pipeline consisted of following steps. 

### a. Declare the necessary libraries to read/write files:
```
from os import listdir   
from os.path import isfile, join  
```
### b. setting the output path and create a folder to store the converted image files:
```
mypath ='test_images/' # calling input path   
output_path ='test_images_output/' # setting output path   

if os.path.exists(output_path) == False:   
    os.mkdir(output_path) # make a folder to store outputs   
```
### c. since the folder contains the multiple files, use for loop statement to import the image file:
```
for n in range(0,len(test_image)): # for statement to read inputs to convert   
```
### d. read out an image using imread function:
```
image = mpimg.imread(join(mypath,test_image[n]))
```
 ![alt text](./test_images/whiteCarLaneSwitch.jpg)
 
### e. extract height and width per image
```
height, width = image.shape[:2]   
```
### f. convert the image to gray scale:
```
image_gray = grayscale(image)   
 
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
 
``` 
 ![alt text](./test_images_output/graywhiteCarLaneSwitch.jpg)
 
### g. gaussian filter and canny function are applied to extract necessary line information:
```
kernel_size = 3   
image_gray_blur = gaussian_blur(image,kernel_size)   
image_gray_blur_canny = canny(image_gray_blur,70,210)   
 
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
 
```    
 ![alt text](./test_images_output/gaussianwhiteCarLaneSwitch.jpg)
 
 ![alt text](./test_images_output/cannywhiteCarLaneSwitch.jpg)
 
### h. determine Region Of Interest (ROI) and extract line information:
```
vertices = np.array([[(50,height),(width/2, height/2+45), (width, height)]], dtype=np.int32)# determine vertice dimension to extract ROI   
    img_roi=region_of_interest(image_gray_blur_canny, vertices)   
    hough_img = hough_lines(img_roi, 1, 1 * np.pi/180, 30, 10, 20)   
    
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x_pos = []
    y_pos = []
    x_neg = []
    y_neg = []
    n_pos = 0
    n_neg = 0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=(y2-y1)/(x2-x1) # calculate the slope of line
            if slope >0: # divide the case, this is for when the slope is positive (lefthand side of line)
                if slope >0.4 and slope < 3: # exclude some noise condition where line is too flat or too steep 
                    x_pos += [x1, x2]
                    y_pos += [y1, y2]
                    n_pos += 1
            else: # separate the case when the slope is negative (righthand side of line)
                if slope <-0.7 and slope >-5:# exclude some noise condition where line is too flat or too steep 
                    x_neg += [x1, x2]
                    y_neg += [y1, y2]
                    n_neg += 1
            
            
      if n_pos !=0 and x_pos != [] and y_pos != [] : # for positive slope case excluding the empty cases where there's no line detected. 
      
        z_pos=np.polyfit(x_pos,y_pos,1) # use polyfit to bring slope (m) and intercept (b)
        f_pos=np.poly1d(z_pos) # functionize, f_pos=m*x+b
        
        min_x_pos=min(x_pos) # calculate min of positive x on interpolated line 
        int_min_x_pos=int(min_x_pos)  #convert it to integer as cv2.line function accepts integer as an input
        min_y_pos=f_pos(min_x_pos) # y position on the interpolated line where x= min(x)
        int_min_y_pos=int(min_y_pos)# convert it to integer
    
        max_x_pos=max(x_pos) # see above for same principle to come up with max of x,y position @ positive line
        int_max_x_pos=int(max_x_pos)
        max_y_pos=f_pos(max_x_pos)
        int_max_y_pos=int(max_y_pos)  
    
        cv2.line(img, (int_min_x_pos,int_min_y_pos), (int_max_x_pos,int_max_y_pos), color, thickness)
 
    if n_neg !=0 and x_neg != [] and y_neg != []:    
            
        z_neg=np.polyfit(x_neg,y_neg,1) 
        f_neg=np.poly1d(z_neg)
    
        min_x_neg=min(x_neg)
        int_min_x_neg=int(min_x_neg)
        min_y_neg=f_neg(min_x_neg)
        int_min_y_neg=int(min_y_neg)
    
        max_x_neg=max(x_neg)
        int_max_x_neg=int(max_x_neg)
        max_y_neg=f_neg(max_x_neg)
        int_max_y_neg=int(max_y_neg)
    
        cv2.line(img, (int_min_x_neg,int_min_y_neg), (int_max_x_neg,int_max_y_neg), color, thickness)
        
        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img  
    
```

 ![alt text](./test_images_output/ROIwhiteCarLaneSwitch.jpg)
  ![alt text](./test_images_output/hough_imgwhiteCarLaneSwitch.jpg)

### i. overlap the extracted line onto the orignal image
```
 result=weighted_img(hough_img,image)   
 plt.imshow(result)   
```
  ![alt text](./test_images_output/weighted_imgwhiteCarLaneSwitch.jpg)

### j. write the image to the designated folder
```
 cv2.imwrite(join(output_path,test_image[n]),result) # writing the image same as the input name
```
---

### k. testing on video clips (1)



<video controls src="./test_videos_output/solidWhiteRight.mp4" />

</video>


### l. testing on video clips (2)

<video controls src="./test_videos_output/solidYellowLeft.mp4" />

</video>

### m. testing on video clips (3)

<video controls src="./test_videos_output/challenge.mp4" />

</video>

## 2. Identify potential shortcomings with your current pipeline

drawline could be vectorized to extract necessary information instead of using nested if statement. Certainly, I need to get used to python script. 


## 3. Suggest possible improvements to your pipeline

ROI determination would be a little better instead of trial and error. 
